In [ ]:
#Pandas, requests, BeautifulSoup used for web scraping
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [ ]:
#Identify user for probing websites
user_agent = "bookworm"

In [ ]:
book_titles = []
authors = []
avg_ratings = []
ratings = []
published_years = []
editions = []
desc = []

#title -> Book Name
#Rating -> Book rating given by the user
#Genre -> Category(Type of book).
#Author -> Book Author
#Desc -> Book description
#url -> Book cover image link

In [ ]:
pages_to_scrape = 10

#Delay between page requests
request_delay = 2

#Iterate all of the pages
for page in range(1, pages_to_scrape + 1):

    # Build the URL for each page
    url = "https://www.goodreads.com/search?page=" + str(page) + "&q=classics&qid=cB8CbxWplQ&search_type=books&tab=books&utf8=✓"

    try:
        #HTTP GET request passed to URL provided the useragent
        headers = {"User-Agent": user_agent}
        response = requests.get(url, headers=headers).text

        #Grab all of the response data using BeautifulSoup
        soup = BeautifulSoup(response, "html5lib")

        #Avoid errors if 404
        if soup.title and "service unavailable" in soup.title.text.lower():
            print(f"Server error on page {page}. Skipping...")
            continue

        #Table that has all of the books listed
        table = soup.find_all("tbody")[0]

        for row in table.find_all("tr"):
            cells = row.find_all("td")[1]

            # get book title
            title = cells.find("a").find("span").text
            book_titles.append(title)

            # get author
            author = cells.find("a", class_="authorName").text
            authors.append(author)


            # rating
            all_ratings = cells.find_all('span', class_ = 'minirating')
            all_ratings_text = all_ratings[0].text.strip()
            pattern_2 = re.compile(r"(\d\.?\d*)\savg")
            avg_ratings.append(pattern_2.search(all_ratings_text).group(1))

            # n_ratings  count how many total ratings
            pattern_4 = re.compile(r"(\d\,?\d*) rating")
            ratings_matches = pattern_4.search(all_ratings_text)
            ratings.append(ratings_matches.group(1) if ratings_matches else 0)


            # published year and handles edge cases for improper format
            year_info = cells.find("span", class_="greyText smallText uitext").text.split()
            year = None
            for item in year_info:
                if item.isdigit() and len(item) == 4:
                    year = item
                    break
            if year:
                published_years.append(year)
            else:
                published_years.append(0)

            #edition information
            edition = cells.find("span", class_="greyText smallText uitext").text.split()[-2]
            editions.append(edition)

        # sleep on delay period
        time.sleep(request_delay)

    except requests.exceptions.RequestException as e:
        # HTTP request errors
        print(f"Error on page {page}: {e}")

    except IndexError as e:
        # list index out of range
        print(f"Index error on page {page}: {e}")

    except Exception as e:
        #handle other unexpected errors
        print(f"Unexpected error on page {page}: {e}")

In [ ]:
# create a dataFrame
data = {
    "Title": book_titles,
    "Author": authors,
    "Average Rating": avg_ratings,
    "Rating": ratings,
    "Year Published": published_years,
    "Edition": editions
}

In [ ]:
goodreads = pd.DataFrame(data)

# Check if there are reviews in the DataFrame and no errors
if not goodreads.empty:
    # Specify the file path where you want to save the CSV file
    csv_file_path = "book_reviews.csv"

    # Save the DataFrame to a CSV file
    goodreads.to_csv(csv_file_path, index=False)

    print(f"CSV file saved to {csv_file_path}")
else:
    print("No reviews to save or an error occurred during scraping.")

    #https://raw.githubusercontent.com/sdhilip200/Content-Based-Recommendation---Good-Reads-data/master/data.csv


CSV file saved to book_reviews.csv
